In [1]:
# google drive mount
from os.path import join
from google.colab import drive

ROOT = '/content/drive'
drive.mount(ROOT)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import os
import sys

DATA_PATH = '/content/drive/MyDrive/GitHub/DL_Study/datasets'
MODULE_PATH = '/content/drive/MyDrive/GitHub/DL_Study/Base'

sys.path.insert(0, DATA_PATH)
sys.path.insert(0, MODULE_PATH)
sys.path

['/content/drive/MyDrive/GitHub/DL_Study/Base',
 '/content/drive/MyDrive/GitHub/DL_Study/datasets',
 '',
 '/content',
 '/env/python',
 '/usr/lib/python37.zip',
 '/usr/lib/python3.7',
 '/usr/lib/python3.7/lib-dynload',
 '/usr/local/lib/python3.7/dist-packages',
 '/usr/lib/python3/dist-packages',
 '/usr/local/lib/python3.7/dist-packages/IPython/extensions',
 '/root/.ipython']

In [3]:
# import
from config import *
from layers import Convolution, Pooling, FullyConnected
from optim import Adam

np.random.seed(42)

# for time series split
!pip install scikit-learn==0.24.2
!pip install impyute

In [4]:
# MSE for Loss
def MSE(y, t):
    return 0.5 * np.mean((y-t)**2)

# ReLU activation function
class ReLU:
    def __init__(self):
        self.mask = None
    
    def forward(self, x):
        self.mask = (x<=0)
        y = x.copy()
        y[self.mask] = 0
        return y

    def backward(self, dy):
        dy[self.mask] = 0
        dx = dy
        return dx

# Loss with ReLU
class ReluWithLoss:
    def __init__(self):
        self.params, self.grads = [], []
        self.activation = ReLU()
        self.cache = None

    def forward(self, x, t):
        N, V = x.shape      # batch, output
        
        x = x.reshape(N, V)
        t = t.reshape(N, V)
        x = self.activation.forward(x)

        loss = MSE(x, t)
        self.cache = (t, x, (N, V))
        return loss

    def backward(self, dy=1):
        t, x, (N, V) = self.cache
        dx = dy * (x-t) / N
        
        dx = self.activation.backward(dx)
        dx = dx.reshape(N, V)
        return dx

In [5]:
# CNN Model with Regression
class CnnModelReg:
    def __init__(self, input_dim = (1, 24, 8), 
                 params={'filter_num':30, 'filter_size':5, 
                         'padding':0, 'stride':1},
                 hidden_size=100, output_size=1):
        filter_num = params['filter_num']
        filter_size = params['filter_size']
        padding = params['padding']
        stride = params['stride']

        # not square
        input_size = input_dim[1]
        input_size2 = input_dim[2]
        conv_output_size_h = (input_size - filter_size + 2*padding)/stride + 1
        conv_output_size_w = (input_size2 - filter_size + 2*padding)/stride + 1
        pool_output_size = int(filter_num*int(conv_output_size_h/2)*int(conv_output_size_w/2))

        self.params = {}
        rand = np.random.randn

        # He initialize
        self.params['cnn_W'] = rand(filter_num, input_dim[0], filter_size, filter_size) / np.sqrt(filter_num/2)
        self.params['cnn_b'] = np.zeros(filter_num)
        self.params['W1'] = rand(pool_output_size, hidden_size)/np.sqrt(pool_output_size/2)
        self.params['b1'] = np.zeros(hidden_size)
        self.params['W2'] = rand(hidden_size, output_size)/np.sqrt(hidden_size/2)
        self.params['b2'] = np.zeros(output_size)

        self.layers=[
                     Convolution(self.params['cnn_W'], self.params['cnn_b'], stride, padding),
                     ReLU(),
                     Pooling(pool_h=2, pool_w=2, stride=2),
                     FullyConnected(self.params['W1'], self.params['b1']),
                     ReLU(),
                     FullyConnected(self.params['W2'], self.params['b2']),
        ]
        self.loss_layer = ReluWithLoss()

        self.grads = []
        self.params = []
        for i in [0, 3, 5]:
            self.params += self.layers[i].params
            self.grads+=self.layers[i].grads

    def predict(self, x):
        x = np.array(x)
        for layer in self.layers:
            x = layer.forward(x)
        return x

    def forward(self, x, t):
        x = np.array(x)
        t = np.array(t)
        x = self.predict(x)
        loss = self.loss_layer.forward(x, t)
        return loss

    def backward(self, dy=1):
        dy = self.loss_layer.backward(dy)

        for layer in reversed(self.layers):
            dy = layer.backward(dy)
        return dy
    
    def fit(self, train_X=None, train_y=None,learning_rate=0.01, epochs=10, batch_size=32, verbose=0):
        optimizer = Adam(learning_rate)

        data_size = train_X.shape[0]
        max_iters = data_size//batch_size

        for epoch in range(1, epochs+1):
            # shuffle train data
            idx = numpy.random.permutation(numpy.arange(data_size))
            x_data = train_X[idx]
            y_data = train_y[idx]

            epoch_loss = 0
            start_time = time.time()
            for iter in range(max_iters):
                batch_x = x_data[iter*batch_size:(iter+1)*batch_size]
                batch_y = y_data[iter*batch_size:(iter+1)*batch_size]

                loss = self.forward(batch_x, batch_y)
                self.backward()
                params, grads = self.params, self.grads
                optimizer.update(params, grads)

                epoch_loss += loss
            avg_loss = epoch_loss/max_iters

            if verbose:
                duration = start_time-time.time()
                print(f'epoch:{epoch}/{epochs}, 시간:{duration:.2f}[s], loss:{avg_loss:.5f}')

In [6]:
# configuration setting
def model_config():
    # parameter for CNN Model
    filter_num = [30]
    filter_size = [3, 5]
    epochs = [30]
    batch_size = [32, 64]
    learning_rate = [0.1, 0.01]
    
    # create config data
    configs = []
    for i in filter_num:
        for j in filter_size:
            for k in epochs:
                for l in batch_size:
                    for m in learning_rate:
                        config = [i, j, k, l, m]
                        configs.append(config)
    return configs

# fucntion for fit cnn model using configs
def model_fit(train_X, train_y, config):
    # unpack config
    n_filter, n_fsize, n_epochs, n_batch, learning_rate = config
    cnn_params = {'filter_num':n_filter, 'filter_size':n_fsize, 
                  'stride':1, 'padding':1}
    model = CnnModelReg(input_dim=(1, 30, 7),params=cnn_params)
    # fit model and return
    model.fit(train_X=train_X, train_y=train_y, epochs=n_epochs, 
              batch_size=n_batch, learning_rate=learning_rate)
    return model

def MAE_metric(x, t):
    return np.mean(numpy.abs(x-t))

def MSE_metric(x, t):
    t = np.array(t)
    return np.mean((x-t)**2)

In [7]:
import pandas as pd
import numpy
import time
from datetime import datetime

numpy.random.seed(42)

data_path = '/content/drive/MyDrive/GitHub/DL_Study/datasets/household_power_consumption/household_power_consumption.txt'
df_parser = lambda x: datetime.strptime(x, '%d/%m/%Y %H:%M:%S')
df = pd.read_csv(data_path, sep=';', parse_dates=[['Date', 'Time']], date_parser=df_parser, index_col=0)
df.head()

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (2,3,4,5,6,7) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Global_active_power,Global_reactive_power,Voltage,Global_intensity,Sub_metering_1,Sub_metering_2,Sub_metering_3
Date_Time,,,,,,,
2006-12-16 17:24:00,4.216,0.418,234.840,18.400,0.000,1.000,17.0
2006-12-16 17:25:00,5.360,0.436,233.630,23.000,0.000,1.000,16.0
2006-12-16 17:26:00,5.374,0.498,233.290,23.000,0.000,2.000,17.0
2006-12-16 17:27:00,5.388,0.502,233.740,23.000,0.000,1.000,17.0
2006-12-16 17:28:00,3.666,0.528,235.680,15.800,0.000,1.000,17.0


In [8]:
df.replace('?', numpy.nan, inplace=True)
# slice
df_slice = df['2007']
df_slice = df_slice.astype(float)
df_slice.isnull().sum()

Global_active_power      3931
Global_reactive_power    3931
Voltage                  3931
Global_intensity         3931
Sub_metering_1           3931
Sub_metering_2           3931
Sub_metering_3           3931
dtype: int64

In [9]:
# mean
from sklearn.impute import SimpleImputer, KNNImputer
from impyute.imputation.cs import mice

imp = SimpleImputer(missing_values = np.nan, strategy='mean')
df_mean = imp.fit_transform(df_slice)
df_mean = pd.DataFrame(df_mean)

# # frequent
# imp = SimpleImputer(strategy='most_frequent')
# df_hotdeck = imp.fit_transform(df_slice)
# df_hotdeck = pd.DataFrame(df_hotdeck)

# # KNN
# imp = KNNImputer(n_neighbors=2, weights='uniform')
# df_knn = imp.fit_transform(df_slice)
# df_knn = pd.DataFrame(df_knn)


# # MICE
# df_mice = mice(df_slice.values)
# df_mice = pd.DataFrame(df_mice)

In [10]:
# task : budapest
# series data to img function
def series_to_img(dataset, time_step=1):
    num = dataset.shape[1]      # features num
    df = pd.DataFrame(dataset)
    cols, names = list(), list()
    # sequence t-n to t-1
    for i in range(time_step, 0, -1):
        cols.append(df.shift(i))
        names += [('var%d(t-%d)' % (j+1, i)) for j in range(num)]

    for i in range(0, 1):
        cols.append(df.shift(-i))
        if i == 0:
            names += [('var%d(t)' % (j+1)) for j in range(num)]
        else:
            names += [('var%d(t+%d)' % (j+1, i)) for j in range(num)]

    agg = pd.concat(cols, axis=1)
    agg.columns = names
    agg.dropna(inplace=True)
    return agg

dataset = df_mean.values
dataset = dataset.astype('float')

n_inputs = 30
n_features = 7
del_idx = n_inputs*n_features+1
del_cols = [i for i in range(del_idx, del_idx+n_features-1)]
new_df = series_to_img(dataset, n_inputs)
new_df.drop(new_df.columns[del_cols], axis=1, inplace=True)
new_df = new_df[:10000]
new_df

,var1(t-30),var2(t-30),var3(t-30),var4(t-30),var5(t-30),var6(t-30),var7(t-30),var1(t-29),var2(t-29),var3(t-29),var4(t-29),var5(t-29),var6(t-29),var7(t-29),var1(t-28),var2(t-28),var3(t-28),var4(t-28),var5(t-28),var6(t-28),var7(t-28),var1(t-27),var2(t-27),var3(t-27),var4(t-27),var5(t-27),var6(t-27),var7(t-27),var1(t-26),var2(t-26),var3(t-26),var4(t-26),var5(t-26),var6(t-26),var7(t-26),var1(t-25),var2(t-25),var3(t-25),var4(t-25),var5(t-25),...,var4(t-6),var5(t-6),var6(t-6),var7(t-6),var1(t-5),var2(t-5),var3(t-5),var4(t-5),var5(t-5),var6(t-5),var7(t-5),var1(t-4),var2(t-4),var3(t-4),var4(t-4),var5(t-4),var6(t-4),var7(t-4),var1(t-3),var2(t-3),var3(t-3),var4(t-3),var5(t-3),var6(t-3),var7(t-3),var1(t-2),var2(t-2),var3(t-2),var4(t-2),var5(t-2),var6(t-2),var7(t-2),var1(t-1),var2(t-1),var3(t-1),var4(t-1),var5(t-1),var6(t-1),var7(t-1),var1(t)
30,2.580,0.136,241.97,10.6,0.0,0.0,0.0,2.552,0.1,241.75,10.4,0.0,0.0,0.0,2.550,0.100,241.64,10.4,0.0,0.0,0.0,2.550,0.100,241.71,10.4,0.0,0.0,0.0,2.554,0.100,241.98,10.4,0.0,0.0,0.0,2.550,0.100,241.83,10.4,0.0,...,11.0,0.0,1.0,0.0,2.660,0.252,241.60,11.0,0.0,1.0,0.0,2.650,0.250,241.14,11.0,0.0,2.0,0.0,2.654,0.250,241.38,11.0,0.0,1.0,0.0,2.642,0.248,240.90,11.0,0.0,2.0,0.0,2.644,0.250,241.15,11.0,0.0,1.0,0.0,2.648
31,2.552,0.100,241.75,10.4,0.0,0.0,0.0,2.550,0.1,241.64,10.4,0.0,0.0,0.0,2.550,0.100,241.71,10.4,0.0,0.0,0.0,2.554,0.100,241.98,10.4,0.0,0.0,0.0,2.550,0.100,241.83,10.4,0.0,0.0,0.0,2.534,0.096,241.07,10.4,0.0,...,11.0,0.0,1.0,0.0,2.650,0.250,241.14,11.0,0.0,2.0,0.0,2.654,0.250,241.38,11.0,0.0,1.0,0.0,2.642,0.248,240.90,11.0,0.0,2.0,0.0,2.644,0.250,241.15,11.0,0.0,1.0,0.0,2.648,0.252,241.32,11.0,0.0,1.0,0.0,2.646
32,2.550,0.100,241.64,10.4,0.0,0.0,0.0,2.550,0.1,241.71,10.4,0.0,0.0,0.0,2.554,0.100,241.98,10.4,0.0,0.0,0.0,2.550,0.100,241.83,10.4,0.0,0.0,0.0,2.534,0.096,241.07,10.4,0.0,0.0,0.0,2.484,0.000,241.29,10.2,0.0,...,11.0,0.0,2.0,0.0,2.654,0.250,241.38,11.0,0.0,1.0,0.0,2.642,0.248,240.90,11.0,0.0,2.0,0.0,2.644,0.250,241.15,11.0,0.0,1.0,0.0,2.648,0.252,241.32,11.0,0.0,1.0,0.0,2.646,0.252,241.35,11.0,0.0,2.0,0.0,2.644
33,2.550,0.100,241.71,10.4,0.0,0.0,0.0,2.554,0.1,241.98,10.4,0.0,0.0,0.0,2.550,0.100,241.83,10.4,0.0,0.0,0.0,2.534,0.096,241.07,10.4,0.0,0.0,0.0,2.484,0.000,241.29,10.2,0.0,0.0,0.0,2.468,0.000,241.23,10.2,0.0,...,11.0,0.0,1.0,0.0,2.642,0.248,240.90,11.0,0.0,2.0,0.0,2.644,0.250,241.15,11.0,0.0,1.0,0.0,2.648,0.252,241.32,11.0,0.0,1.0,0.0,2.646,0.252,241.35,11.0,0.0,2.0,0.0,2.644,0.252,241.38,11.0,0.0,1.0,0.0,2.656
34,2.554,0.100,241.98,10.4,0.0,0.0,0.0,2.550,0.1,241.83,10.4,0.0,0.0,0.0,2.534,0.096,241.07,10.4,0.0,0.0,0.0,2.484,0.000,241.29,10.2,0.0,0.0,0.0,2.468,0.000,241.23,10.2,0.0,0.0,0.0,2.486,0.000,242.18,10.2,0.0,...,11.0,0.0,2.0,0.0,2.644,0.250,241.15,11.0,0.0,1.0,0.0,2.648,0.252,241.32,11.0,0.0,1.0,0.0,2.646,0.252,241.35,11.0,0.0,2.0,0.0,2.644,0.252,241.38,11.0,0.0,1.0,0.0,2.656,0.256,242.03,11.0,0.0,1.0,0.0,2.650
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10025,1.728,0.000,241.81,7.2,0.0,0.0,18.0,1.720,0.0,242.65,7.0,0.0,0.0,18.0,1.738,0.000,242.62,7.2,0.0,0.0,18.0,1.686,0.000,242.38,7.0,0.0,0.0,18.0,1.626,0.000,242.41,6.6,0.0,0.0,18.0,1.612,0.000,242.81,6.6,0.0,...,7.0,0.0,4.0,18.0,1.716,0.252,242.90,7.0,0.0,4.0,18.0,1.710,0.240,242.67,7.0,0.0,3.0,18.0,1.722,0.256,243.53,7.0,0.0,4.0,18.0,1.752,0.268,243.78,7.2,0.0,3.0,18.0,1.728,0.330,243.66,7.2,0.0,3.0,18.0,1.586
10026,1.720,0.000,242.65,7.0,0.0,0.0,18.0,1.738,0.0,242.62,7.2,0.0,0.0,18.0,1.686,0.000,242.38,7.0,0.0,0.0,18.0,1.626,0.000,242.41,6.6,0.0,0.0,18.0,1.612,0.000,242.81,6.6,0.0,0.0,18.0,1.602,0.000,241.99,6.6,0.0,...,7.0,0.0,4.0,18.0,1.710,0.240,242.67,7.0,0.0,3.0,18.0,1.722,0.256,243.53,7.0,0.0,4.0,18.0,1.752,0.268,243.78,7.2,0.0,

In [12]:
import numpy
import time
from sklearn.model_selection import TimeSeriesSplit
from sklearn.preprocessing import MinMaxScaler

n_splits = 10
train_test_split = TimeSeriesSplit(n_splits=n_splits+1, gap=n_inputs).split(new_df)
next(train_test_split)

configs = model_config()
history = []
best_error = []
i = 1

print('config : filter_num, filter_size, epochs, batch_size, learning_rate')

# neted cross validation
for train_cv_indices, test_cv_indices in train_test_split:
    print(f'fold : {i}/{n_splits}')
    i+=1

    # split x, y data
    train_cv_X, train_cv_y = new_df.iloc[train_cv_indices, :-1].values, new_df.iloc[train_cv_indices,-1].values
    test_cv_X, test_cv_y = new_df.iloc[test_cv_indices, :-1].values, new_df.iloc[test_cv_indices, -1].values

    # length for validation set
    test_length = len(test_cv_X)

    # scaling data
    scaler_x = MinMaxScaler()
    train_cv_X = scaler_x.fit_transform(train_cv_X)
    test_cv_X = scaler_x.transform(test_cv_X)

    train_X, val_X = train_cv_X[:-test_length, :], train_cv_X[-test_length:, :]
    train_y, val_y = train_cv_y[:-test_length], train_cv_y[-test_length:]

    # reshape
    # inner loop
    train_X = train_X.reshape(-1, 1, n_inputs, n_features)
    val_X = val_X.reshape(-1, 1, n_inputs, n_features)
    train_y = train_y.reshape(-1, 1)
    val_y = val_y.reshape(-1, 1)

    # outer loop
    train_cv_X = train_cv_X.reshape(-1, 1, n_inputs, n_features)
    test_cv_X = test_cv_X.reshape(-1, 1, n_inputs, n_features)
    train_cv_y = train_cv_y.reshape(-1, 1)
    test_cv_y = test_cv_y.reshape(-1, 1)

    # model fit, inner
    errors = []
    for idx, cfg in enumerate(configs):
        print(f' == train {cfg} model == ', end=' ')
        model = model_fit(train_X, train_y, cfg)
        predicted = model.predict(val_X)
        error = np.sqrt(MSE_metric(predicted, val_y))   # rmse
        print(f'error(rmse):{error.item():.2f}')
        if errors:
            if error < min(errors):
                param = idx
        else:
            param = idx
        errors.append(error)

    history.append(errors)

    # outer
    selected_model = model_fit(train_cv_X,train_cv_y, configs[param])
    predicted = selected_model.predict(test_cv_X)
    error = np.sqrt(MSE_metric(predicted, test_cv_y))
    best_error.append(error)

    # model eval
    print(f'best_model => error(rmse) : {error.item():.2f}, param:{configs[param]}')

config : filter_num, filter_size, epochs, batch_size, learning_rate
fold : 1/10
 == train [30, 3, 30, 32, 0.1] model ==  error(rmse):7.31
 == train [30, 3, 30, 32, 0.01] model ==  error(rmse):8.67
 == train [30, 3, 30, 64, 0.1] model ==  error(rmse):5.09
 == train [30, 3, 30, 64, 0.01] model ==  error(rmse):1.42
 == train [30, 5, 30, 32, 0.1] model ==  error(rmse):1.88
 == train [30, 5, 30, 32, 0.01] model ==  error(rmse):2.19
 == train [30, 5, 30, 64, 0.1] model ==  error(rmse):135.76
 == train [30, 5, 30, 64, 0.01] model ==  error(rmse):0.47
best_model => error(rmse) : 3.68, param:[30, 5, 30, 64, 0.01]
fold : 2/10
 == train [30, 3, 30, 32, 0.1] model ==  error(rmse):4.61
 == train [30, 3, 30, 32, 0.01] model ==  error(rmse):2.17
 == train [30, 3, 30, 64, 0.1] model ==  error(rmse):7.44
 == train [30, 3, 30, 64, 0.01] model ==  error(rmse):4.07
 == train [30, 5, 30, 32, 0.1] model ==  error(rmse):154.48
 == train [30, 5, 30, 32, 0.01] model ==  error(rmse):1.57
 == train [30, 5, 30, 6

In [13]:
model_evaluation = sum(best_error)
model_evaluation /= n_splits
print(f'evaluation [Mean RMSE] : {model_evaluation}')

evaluation [Mean RMSE] : 16.701108026312955


In [20]:
predicted = selected_model.predict(test_cv_X)
print(f'MSE : {MSE_metric(predicted, test_cv_y)}')
print(f'RMSE : {np.sqrt(MSE_metric(predicted, test_cv_y))}')

def MAE_metric(x, t):
    t = np.array(t)
    return np.mean(numpy.absolute(x-t))
print(f'MAE : {MAE_metric(predicted, test_cv_y)}')

MSE : 350.1154925931125
RMSE : 18.71137334866451
MAE : 18.128321765963225
